In [ ]:
# Utilizado para remover os warnings
import warnings
warnings.filterwarnings('ignore')
# Usado para plotar na célula ao invés de abrir uma nova janela
%matplotlib inline

In [ ]:
# Atenção: instalar o pacote: pip install factor_analyzer
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from factor_analyzer import FactorAnalyzer
from factor_analyzer.factor_analyzer import calculate_kmo

In [ ]:
# Obter os dados 
df_fundamental = pd.read_excel("df_inep.xlsx")
df_fundamental = df_fundamental.iloc[:, [7,9,11,13,15, 17]]

In [ ]:
# Criar amostra aleatoria com 300 valores
tamanho_amostra = 300
df_fundamental_amostra = df_fundamental.sample(n=tamanho_amostra, replace=False)

In [ ]:
# Verificando o dataframe
df_fundamental_amostra.head()

In [ ]:
# Gerando um gráfico de correlações com pairplot
df = df_fundamental_amostra
# Calcular a matriz de correlação
corr_matrix = df.corr()
# Criar um Pair Plot com distribuições
sns.set(style="ticks")
g = sns.pairplot(df, diag_kind="kde", markers="o")
# Tamanho da fonte para as anotações
fontsize = 16 
# Adicionar anotações de correlação nas células dos gráficos de dispersão de pares
for i, (ax_row, ax_col) in enumerate(zip(g.axes, g.axes.T)):
    for j, ax in enumerate(ax_col):
        if i != j:
            corr_value = corr_matrix.iloc[i, j]
            ax.annotate(f"corr = {corr_value:.2f}", xy=(0.5, 0.95), xycoords='axes fraction',
                        ha='center', fontsize=fontsize, color='black', backgroundcolor='white')
            
            # Adicionar uma linha de regressão linear
            sns.regplot(data=df, x=df.columns[i], y=df.columns[j], ax=ax, scatter=False, color='blue')

g.fig.suptitle("Pair Plot com Distribuições, Correlações e Linhas de Regressão", y=1.02)
plt.show()

In [ ]:
# Gerando o gráfico de calor para analisar a correlação entre as variáveis
df = df_fundamental_amostra
# Calcular a matriz de correlação
corr_matrix = df.corr()
# Configurar o estilo do gráfico
sns.set(style="white")
# Criar um gráfico de calor de correlação
plt.figure(figsize=(10, 8))
sns.heatmap(corr_matrix, annot=True, fmt=".2f", cmap='coolwarm', linewidths=.5)
# Configurar título e rótulos
plt.title("Gráfico de Calor de Correlação")
plt.xlabel("")
plt.ylabel("")
plt.show()

In [ ]:
# Aplicando a estatística KMO
df = df_fundamental_amostra
# Calcular a matriz de correlação
rho = df.corr()
max_length = 10
# Realize o teste KMO
kmo_per_variable, kmo_total = calculate_kmo(rho)
# Imprima os resultados
for i, kmo in enumerate(kmo_per_variable):
    variable_name = rho.columns[i]
    # Formate o nome da variável com espaços em branco adicionados para atingir o comprimento máximo
    formatted_variable_name = f"{variable_name:{max_length}}"
    print(f"KMO {formatted_variable_name}: {kmo:.2f}")
# Imprima o KMO total
print(f"KMO Total: {kmo_total:.2f}")

In [ ]:
# Aplicando o teste de Bartlett
df = df_fundamental_amostra
# Calcular o teste de esfericidade de Bartlett
estatistica, p_valor = bartlett(*[df[col] for col in df.columns])
# Definir um nível de significância
nivel_significancia = 0.05
# Verificar a hipótese nula
resultado = ''
if p_valor < nivel_significancia:
    resultado = 'Rejeitamos a hipótese nula. A matriz de correlação não é uma matriz de identidade.'
else:
    resultado = 'Não rejeitamos a hipótese nula. A matriz de correlação é uma matriz de identidade.'
    
print(f'{resultado}\nEstatística X²: {estatistica}\np-value: {p_valor}')


In [ ]:
# Teste de esfericidade de Bartlett-Mauchly
import pingouin as pg
df = df_fundamental_amostra
# Execute o teste de esfericidade de Bartlett-Mauchly
result = pg.sphericity(df)
# Imprima o resultado
print(result)